In [1]:
from dataclasses import dataclass
from dataclasses import field
from enum import Enum
from enum import IntEnum
import functools
import itertools
import math
import numpy as np
import random
import re
import typing

In [2]:
def res(day, level, result):
    print(f'Result of [Day {day}, Level {level}]: {result}')

In [3]:
with open('../data/11_input_test.txt') as f:
    test_input = f.read()

with open('../data/11_input.txt') as f:
    input = f.read()

def process(content):
    data = [[ch for ch in line] for line in content.split('\n') if len(line) > 0]
    return np.array(data)

test_data = process(test_input)
data = process(input)

floor = '.'
free = 'L'
occupied = '#'

In [4]:
def step1(arr):
    next = arr.copy()
    for (row, col), val in np.ndenumerate(arr):
        from_row = max(row-1, 0)
        from_col = max(col-1, 0)
        neighborsAndCenter = arr[from_row:row+2, from_col:col+2]
        occupiedNeighbors = (neighborsAndCenter == occupied).sum() - int(val == occupied)
        nextState = val
        if(val == free):
            if(occupiedNeighbors == 0):
                nextState = occupied            
        elif(val == occupied):
            if(occupiedNeighbors >= 4):
                nextState = free
        next[row, col] = nextState
    return next

def level1(arr):
    prev = None
    cur = arr
    while(not np.array_equal(prev, cur)):
        prev = cur
        cur = step1(prev)
    return cur

Haha, wrong version implemented :D
The following version does not lose vision once a seat (either L or #) is seen, but continues till the end of the array

In [5]:
"""def see(arr):
    return int((arr == occupied).sum() >= 1)

def step2(arr):
    next = arr.copy()

    num_rows, num_cols = arr.shape

    for (row, col), val in np.ndenumerate(arr):
        col_above = see(arr[:row,col])
        col_below = see(arr[row+1:,col])
        row_left = see(arr[row,:col])
        row_right = see(arr[row,col+1:])
        len = min(row, col)
        diag_top_left = see(arr[list(itertools.islice(dec_gen(row), len)), list(itertools.islice(dec_gen(col), len))])
        len = min(row, num_cols - col - 1)
        diag_top_right = see(arr[list(itertools.islice(dec_gen(row), len)), list(itertools.islice(inc_gen(col), len))])
        len = min(num_rows - row - 1, num_cols - col - 1)
        diag_bottom_right = see(arr[list(itertools.islice(inc_gen(row), len)), list(itertools.islice(inc_gen(col), len))])
        len = min(num_rows - row - 1, col)
        diag_bottom_left = see(arr[list(itertools.islice(inc_gen(row), len)), list(itertools.islice(dec_gen(col), len))])
        occupiedNeighbors = col_above + col_below + row_left + row_right + diag_top_left + diag_top_right + diag_bottom_right + diag_bottom_left

        nextState = val
        if(val == free):
            if(occupiedNeighbors == 0):
                nextState = occupied            
        elif(val == occupied):
            if(occupiedNeighbors >= 5):
                nextState = free
        next[row, col] = nextState
    return next"""

'def see(arr):\n    return int((arr == occupied).sum() >= 1)\n\ndef step2(arr):\n    next = arr.copy()\n\n    num_rows, num_cols = arr.shape\n\n    for (row, col), val in np.ndenumerate(arr):\n        col_above = see(arr[:row,col])\n        col_below = see(arr[row+1:,col])\n        row_left = see(arr[row,:col])\n        row_right = see(arr[row,col+1:])\n        len = min(row, col)\n        diag_top_left = see(arr[list(itertools.islice(dec_gen(row), len)), list(itertools.islice(dec_gen(col), len))])\n        len = min(row, num_cols - col - 1)\n        diag_top_right = see(arr[list(itertools.islice(dec_gen(row), len)), list(itertools.islice(inc_gen(col), len))])\n        len = min(num_rows - row - 1, num_cols - col - 1)\n        diag_bottom_right = see(arr[list(itertools.islice(inc_gen(row), len)), list(itertools.islice(inc_gen(col), len))])\n        len = min(num_rows - row - 1, col)\n        diag_bottom_left = see(arr[list(itertools.islice(inc_gen(row), len)), list(itertools.islice(dec

In [6]:
def inc_gen(x):
    while True:
        x = x+1
        yield x

def dec_gen(x):
    while True:
        x = x-1
        yield x

def see(arr):
    for x in arr:
        if(x == free):
            return 0
        elif(x == occupied):
            return 1
    return 0

def step2(arr):
    next = arr.copy()

    num_rows, num_cols = arr.shape

    for (row, col), val in np.ndenumerate(arr):
        col_above = see(np.flip(arr[:row,col]))
        col_below = see(arr[row+1:,col])
        row_left = see(np.flip(arr[row,:col]))
        row_right = see(arr[row,col+1:])
        len = min(row, col)
        diag_top_left = see(arr[list(itertools.islice(dec_gen(row), len)), list(itertools.islice(dec_gen(col), len))])
        len = min(row, num_cols - col - 1)
        diag_top_right = see(arr[list(itertools.islice(dec_gen(row), len)), list(itertools.islice(inc_gen(col), len))])
        len = min(num_rows - row - 1, num_cols - col - 1)
        diag_bottom_right = see(arr[list(itertools.islice(inc_gen(row), len)), list(itertools.islice(inc_gen(col), len))])
        len = min(num_rows - row - 1, col)
        diag_bottom_left = see(arr[list(itertools.islice(inc_gen(row), len)), list(itertools.islice(dec_gen(col), len))])
        occupiedNeighbors = col_above + col_below + row_left + row_right + diag_top_left + diag_top_right + diag_bottom_right + diag_bottom_left

        nextState = val
        if(val == free):
            if(occupiedNeighbors == 0):
                nextState = occupied            
        elif(val == occupied):
            if(occupiedNeighbors >= 5):
                nextState = free
        next[row, col] = nextState
    return next

def level2(arr):
    prev = None
    cur = arr
    while(not np.array_equal(prev, cur)):
        prev = cur
        cur = step2(prev)
    return cur

In [7]:
# Test input - Level 1
res(11, 'Test-1', (level1(test_data) == '#').sum())

Result of [Day 11, Level Test-1]: 37


In [8]:
# Level 1 
res(11, 1, (level1(data) == '#').sum())

Result of [Day 11, Level 1]: 2113


In [9]:
# Test input - Level 2 
res(11, 'Test-2', (level2(test_data) == '#').sum())

Result of [Day 11, Level Test-2]: 26


In [10]:
# Level 2 
res(11, 2, (level2(data) == '#').sum())

Result of [Day 11, Level 2]: 1865
